In [2]:
import v20
import pandas as pd
import defs

In [3]:
# create context
ctx = v20.Context(hostname='api-fxpractice.oanda.com', token=defs.key)
ctx.set_header(key='Authorization', value=defs.key)

# streaming context
ctxs = v20.Context(hostname='stream-fxpractice.oanda.com', token=defs.key)
ctxs.set_header(key='Authorization', value=defs.key)

In [4]:
instruments = ctx.account.instruments(defs.ACCOUNT_ID).get('instruments')
insts = {i.name:i.dict() for i in instruments}
insts

ResponseNoField: 400 response for GET https://api-fxpractice.oanda.com:443/v3/accounts//instruments does not have field 'instruments' (contains 'errorCode', 'errorMessage')

In [ ]:
inst_str=','.join(insts.keys())
inst_str

In [ ]:
%%time
prices = ctx.pricing.get(defs.ACCOUNT_ID, instruments=inst_str).get('prices')
price_list = []
for p in prices:
    price_list.append(dict(
        instrument=p.instrument,
        bid=p.bids[0].price,
        ask=p.asks[0].price,
        spread =p.asks[0].price-p.bids[0].price))
df = pd.DataFrame(price_list)    
print(df[df.spread < 0.0003].sort_values(by='spread').to_string(index=False))

In [ ]:
insts['EUR_CAD']